In [128]:
# Import Data
import pandas as pd
import numpy as np
dataFrame = pd.read_excel("CRP2021_raw_data.xlsx")
# Delete 2nd row (question)
question_title_row = list(dataFrame.iloc[0])
data = dataFrame.drop([0])

['Cooking Slovene national dishes',
 'Reading Slovene books',
 'Following current affairs from Slovenia',
 'Attending activities in Slovenian communities and organizations',
 'Knowledge of Slovenian language',
 'Visits to Slovenia',
 'Promotion of Slovenia to the country of my current residence',
 'Preserving Slovenian traditions',
 'Knowledge of Slovenian history',
 'Rooting for Slovenian sports teams',
 'Buying and using Slovenian products',
 'Gender:',
 'Year of birth:',
 'Country of current residence',
 'Were you born in Slovenia?',
 'In which year did you move out of Slovenia?',
 'What is your personal status?',
 'Do you have children?',
 'What is the highest level of education that you have completed so far?',
 'Field of education',
 'Other:',
 'Current employment status',
 'Other:',
 'VTIS – Društvo v tujini izobraženih Slovencev',
 'ASEF – American Slovenian Education Foundation',
 'Mlada akademija – Društvo doktorskih študentov ali raziskovalcev na začetku kariere',
 'SMUL – S

# Preprocessing data

In [105]:
# remove qualitative data (save NLP for later if time)
data = data.drop(columns=["Q10_11_text", "Q11_9_text", "Q12l_text", "Q13_6_text", "Q17_7_text", "Q19g_text", "Q67_7_text", "Q21g_text", "Q23h_text", "Q24a", "Q24b", "Q24c", "Q24d", "Q26k_text", "Q27a", "Q27b", "Q27c", "Q27d", "Q29h_text", "Q30a", "Q30b", "Q30c", "Q30d", "Q32j_text", "Q33a", "Q33b", "Q33c", "Q33d", "Q40a", "Q43h_text", "Q44a", "Q44b", "Q46l_text", "Q47a", "Q47b", "Q50a", "Q50b", "Q50c", "Q50d", "Q51a", "Q51b", "Q51c", "Q51d", "Q52", "Q53", "Q53", "Q55l_text", "Q56", "Q58_2_text", "Q59l_text", "Q60", "Q62l_text", "Q64l_text", "Q65"])

In [106]:
# Parse all data as numbers
for column in data.columns:
    data[column] = pd.to_numeric(data[column])
# Replace negative numbers with null values
data = data.clip(lower=0,)
data.shape

(200, 216)

In [ ]:
# Flatten multi-select columns into one CSV column
# relate discrete data with survey answers
# Will code this once data analysis starts

# Finding correlations in data

In [107]:
# calculate the covariance matrix
# there are 216 variables, so this will be of shape (216, 216). Each point is how much one variable correlates to another (scale between -1 and 1 inclusive).
# The main diagonal corresponds to the correlation of each variable to itself, so all values will be 1.
# The higher the absolute value (outside the main diagonal), the higher the chance of a meaningful variable causation
cov_mat = np.corrcoef(data, rowvar=False)
cov_mat

/Users/carolinemorton/Library/Caches/pypoetry/virtualenvs/crp2021-bpNPsJmt-py3.11/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/carolinemorton/Library/Caches/pypoetry/virtualenvs/crp2021-bpNPsJmt-py3.11/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


array([[ 1.        ,  0.22701095, -0.00574449, ...,  0.05551306,
         0.0462033 ,  0.0114151 ],
       [ 0.22701095,  1.        ,  0.16645008, ...,  0.10126657,
         0.08311889,  0.00826898],
       [-0.00574449,  0.16645008,  1.        , ...,  0.01236705,
         0.03320435, -0.05161398],
       ...,
       [ 0.05551306,  0.10126657,  0.01236705, ...,  1.        ,
         0.46045551, -0.05188063],
       [ 0.0462033 ,  0.08311889,  0.03320435, ...,  0.46045551,
         1.        ,  0.08866281],
       [ 0.0114151 ,  0.00826898, -0.05161398, ..., -0.05188063,
         0.08866281,  1.        ]])

In [140]:
# Find the highest correlations

best_correlations = []
seen_pairs = []
for row_index, row in enumerate(cov_mat):
    for col_index, score in enumerate(row):
        if col_index == row_index:
            # we are on the main diagonal, so ignore this element
            continue
            
        if np.isnan(score):
            # Some scores are nan (e.g. if a text variable is involved)
            continue
            
        row_name = data.columns[row_index]
        row_desc = question_title_row[row_index]
        if isinstance(row_desc, str):
            row_name +=  " " +row_desc
        col_name = data.columns[col_index]
        col_desc = question_title_row[col_index]
        if isinstance(col_desc, str):
            col_name += " " + col_desc
        
        skip = False
        for b1, b2, _ in best_correlations:
            if (row_name == b1 and col_name == b2) or (row_name == b2 and col_name == b1):
                # deduplicate data points
                skip = True
                break
        if skip:
            continue
            
        best_correlations.append((row_name, col_name, np.abs(score)))
        
best_correlations_dtype = [('between_1', "U300"), ("between_2", "U300"), ("score", float)]
best_correlations = np.array(best_correlations, dtype=best_correlations_dtype)
best_correlations = np.sort(best_correlations, order="score")
best_correlations = np.flip(best_correlations)
best_correlations

array([('Q36h_text Other:', 'Q39i_text Other:', 1.00000000e+00),
       ('Q29c By providing adequate information support about the possibilities of collaboration options for Slovenian intellectuals abroad (e.g., in the field of science and education).', 'Q35 Do you think that Slovenia works enough to promote collaboration and networking of Slovenian cultural figures abroad?', 9.89428879e-01),
       ('Q24d Vpišite besedilo', 'Q26h By supporting the academic exchange of knowledge, for example by financing joint research of Slovenian and foreign universities, opening up the possibilities of guest lectures at Slovenian universities etc.', 9.74679602e-01),
       ...,
       ('Q1k Buying and using Slovenian products', 'Q23c By providing adequate information support about the possibilities of collaboration and networking of Slovenians abroad on different fields.', 1.74346269e-19),
       ('Q12l_text Other organizations in my country of residence that connect Slovenians abroad. Which one?', 